In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import gc

In [3]:
os.chdir('/home/luke/Desktop/kaggle/Home_Credit_Default_Risk')
x_val = pd.read_csv('x_val.csv')
y_val = pd.read_csv('y_val.csv')
partial_y_train = pd.read_csv('partial_y_train.csv')
partial_x_train = pd.read_csv('partial_x_train.csv')

In [4]:
print('Training data shape: ', partial_x_train.shape)
print('Training label shape: ', partial_y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation label shape: ', y_val.shape)

Training data shape:  (239999, 239)
Training label shape:  (239999, 1)
Validation data shape:  (67510, 239)
Validation label shape:  (67510, 1)


In [4]:
partial_x_train.head()

,2.197455179324805385e-01,0.000000000000000000e+00,1.000000000000000000e+00,0.000000000000000000e+00.1,5.263157894736841813e-02,2.454384230388970047e-03,5.693483146067415679e-02,5.533520533520533413e-02,4.377104377104376609e-02,1.560552770777368613e-02,...,0.000000000000000000e+00.154,0.000000000000000000e+00.155,0.000000000000000000e+00.156,0.000000000000000000e+00.157,0.000000000000000000e+00.158,0.000000000000000000e+00.159,0.000000000000000000e+00.160,0.000000000000000000e+00.161,0.000000000000000000e+00.162,0.000000000000000000e+00.163
0,0.219751,0.0,0.0,0.0,0.000000,0.002089,0.126730,0.094068,0.108866,0.157398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.219757,0.0,0.0,1.0,0.105263,0.001166,0.056180,0.061495,0.057239,0.093190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.219760,0.0,0.0,1.0,0.052632,0.000935,0.236725,0.158301,0.217733,0.134897,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.219762,0.0,1.0,1.0,0.000000,0.002281,0.388515,0.158915,0.346801,0.129705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.219765,0.0,1.0,1.0,0.000000,0.001320,0.119333,0.098333,0.098765,0.309272,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [22]:
x_test = pd.read_csv('x_test.csv')

In [23]:
x_test_use = np.array(x_test)

In [79]:
x_test_use.shape

(48743, 239)

In [83]:
partial_x_train.head()

,2.197455179324805385e-01,0.000000000000000000e+00,1.000000000000000000e+00,0.000000000000000000e+00.1,5.263157894736841813e-02,2.454384230388970047e-03,5.693483146067415679e-02,5.533520533520533413e-02,4.377104377104376609e-02,1.560552770777368613e-02,...,0.000000000000000000e+00.154,0.000000000000000000e+00.155,0.000000000000000000e+00.156,0.000000000000000000e+00.157,0.000000000000000000e+00.158,0.000000000000000000e+00.159,0.000000000000000000e+00.160,0.000000000000000000e+00.161,0.000000000000000000e+00.162,0.000000000000000000e+00.163
0,0.219751,0.0,0.0,0.0,0.000000,0.002089,0.126730,0.094068,0.108866,0.157398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.219757,0.0,0.0,1.0,0.105263,0.001166,0.056180,0.061495,0.057239,0.093190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.219760,0.0,0.0,1.0,0.052632,0.000935,0.236725,0.158301,0.217733,0.134897,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.219762,0.0,1.0,1.0,0.000000,0.002281,0.388515,0.158915,0.346801,0.129705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.219765,0.0,1.0,1.0,0.000000,0.001320,0.119333,0.098333,0.098765,0.309272,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [84]:
x_test.head()

,-2.806993906034893627e-06,0.000000000000000000e+00,0.000000000000000000e+00.1,1.000000000000000000e+00,0.000000000000000000e+00.2,9.348203259945449956e-04,1.307865168539325829e-01,7.388557388557388805e-02,1.021324354657687922e-01,2.569996399789525876e-01,...,0.000000000000000000e+00.158,0.000000000000000000e+00.159,0.000000000000000000e+00.160,0.000000000000000000e+00.161,0.000000000000000000e+00.162,0.000000000000000000e+00.163,1.000000000000000000e+00.15,0.000000000000000000e+00.164,1.000000000000000000e+00.16,0.000000000000000000e+00.165
0,0.000008,0.0,0.0,1.0,0.000000,0.000627,0.044387,0.061443,0.034792,0.491595,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000031,0.0,1.0,1.0,0.000000,0.001512,0.154373,0.265830,0.147026,0.260475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000073,0.0,0.0,1.0,0.105263,0.002474,0.382022,0.184872,0.382716,0.361433,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.000101,0.0,1.0,0.0,0.052632,0.001320,0.144944,0.118761,0.145903,0.134897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000112,0.0,1.0,1.0,0.000000,0.002089,0.228387,0.128642,0.191919,0.344429,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Format the training and testing data 
train = np.array(application_train.drop(columns = 'TARGET'))
test = np.array(application_test)

train_labels = np.array(train_labels).reshape((-1, ))

# 10 fold cross validation
folds = KFold(n_splits=5, shuffle=True, random_state=50)

# Validation and test predictions
valid_preds = np.zeros(train.shape[0])
test_preds = np.zeros(test.shape[0])

# Iterate through each fold
for n_fold, (train_indices, valid_indices) in enumerate(folds.split(train)):
    # Training data for the fold
    train_fold, train_fold_labels = train[train_indices, :], train_labels[train_indices]
    
    # Validation data for the fold
    valid_fold, valid_fold_labels = train[valid_indices, :], train_labels[valid_indices]
    
    # LightGBM classifier with hyperparameters
    clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.1,
        subsample=.8,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2
    )
    
    # Fit on the training data, evaluate on the validation data
    clf.fit(train_fold, train_fold_labels, 
            eval_set= [(train_fold, train_fold_labels), (valid_fold, valid_fold_labels)], 
            eval_metric='auc', early_stopping_rounds=100, verbose = False
           )
    
    # Validation preditions
    valid_preds[valid_indices] = clf.predict_proba(valid_fold, num_iteration=clf.best_iteration_)[:, 1]
    
    # Testing predictions
    test_preds += clf.predict_proba(test, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    # Display the performance for the current fold
    print('Fold %d AUC : %0.6f' % (n_fold + 1, roc_auc_score(valid_fold_labels, valid_preds[valid_indices])))
    
    # Delete variables to free up memory
    del clf, train_fold, train_fold_labels, valid_fold, valid_fold_labels
    gc.collect()
    

# Make a submission dataframe
#submission = application_test[['SK_ID_CURR']]
#submission['TARGET'] = test_preds


In [44]:
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier

In [13]:
partial_x_train.shape

(239999, 239)

In [38]:
def test(unit,epochs,batch_size):
    model = models.Sequential()
    model.add(layers.Dense(64, input_shape=(239,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(partial_x_train,partial_y_train, epochs=20, verbose=0,batch_size=100,validation_data=(x_val, y_val))
    y_pred = model.predict_proba(x_test_use)
    return roc_auc_score(y_val, model.predict_proba(x_val))

In [39]:
from sklearn.model_selection import GridSearchCV

In [68]:
%%time
# Function to create model, required for KerasClassifier
def create_model(optimizer, init):
    model = models.Sequential()
    model.add(layers.Dense(64, kernel_initializer=init, input_shape=(239,)))
    model.add(layers.Dense(64, kernel_initializer=init, activation='relu'))
    model.add(layers.Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    return(model)
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['normal']
epochs = [5,10,20,25,50,100,200]
batches = [100,200,400,500,600,700]
param_grid = dict(optimizer = optimizers, init = init,epochs = epochs,batch_size = batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc')
grid_result = grid.fit(partial_x_train, partial_y_train)

CPU times: user 15h 14min 40s, sys: 48min 57s, total: 16h 3min 37s
Wall time: 11h 16min 20s


In [74]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.742933 using {'batch_size': 400, 'epochs': 20, 'init': 'normal', 'optimizer': 'adam'}
0.723196 (0.005718) with: {'batch_size': 100, 'epochs': 5, 'init': 'normal', 'optimizer': 'rmsprop'}
0.741225 (0.003531) with: {'batch_size': 100, 'epochs': 5, 'init': 'normal', 'optimizer': 'adam'}
0.722961 (0.005566) with: {'batch_size': 100, 'epochs': 10, 'init': 'normal', 'optimizer': 'rmsprop'}
0.741018 (0.003982) with: {'batch_size': 100, 'epochs': 10, 'init': 'normal', 'optimizer': 'adam'}
0.719698 (0.005179) with: {'batch_size': 100, 'epochs': 20, 'init': 'normal', 'optimizer': 'rmsprop'}
0.739504 (0.002853) with: {'batch_size': 100, 'epochs': 20, 'init': 'normal', 'optimizer': 'adam'}
0.717166 (0.002190) with: {'batch_size': 100, 'epochs': 25, 'init': 'normal', 'optimizer': 'rmsprop'}
0.738858 (0.003820) with: {'batch_size': 100, 'epochs': 25, 'init': 'normal', 'optimizer': 'adam'}
0.719218 (0.008245) with: {'batch_size': 100, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.

In [71]:
from keras import regularizers

In [86]:
model = models.Sequential()
model.add(layers.Dense(64, kernel_initializer='normal',input_shape=(239,)))
model.add(layers.Dense(64, kernel_initializer='normal',activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [93]:
#model.fit(partial_x_train,partial_y_train,epochs = 20,batch_size=400,validation_data = (x_val,y_val),verbose = 1)


In [97]:
roc_auc_score(y_val, model.predict_proba(x_val))

0.7409343871448655